### Introduction to PySpack

In this notebook we are going to go with the basics of PySpacks. First you need to run the following code cell and make sure that you have the spacks context.

In [5]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark


import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)
sc

<SparkContext master=local[*] appName=YourTest>

Let's create `1000` numbers using the range function.

In [6]:
nums = list(range(0, 100000))
len(nums)

100000

We can use our spacks to do distributed computing by creating something called an rdd. The following code will create a `Resilient Distributed Dataset (RDD)`

In [7]:
nums_rdd = sc.parallelize(nums)
nums_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

So, how can we get the data that is in this RDD. We can use the method called `collect()` which is not efficient but however it takes all the data in the dataset.

In [8]:
nums_rdd.collect()[:7]

[0, 1, 2, 3, 4, 5, 6]

The method called `take` allows us to take certain number of samples in the dataset

In [9]:
nums_rdd.take(3)

[0, 1, 2]

We can use the method called `map` which maps through the elements of the dataset and returns a modified version of the data. Here is an example that square the numbers that are in the rdd.

In [10]:
squared_rdd = nums_rdd.map(lambda x: x*x)
squared_rdd.take(3)

[0, 1, 4]

Here is a second example that returns a number after it has been squared and the the total number of digits it has as a python tuple.

In [11]:
total_digits_rdd = squared_rdd.map(lambda x: (x, len(str(x))))
total_digits_rdd.take(3)

[(0, 1), (1, 1), (4, 1)]

We can use the filter method to remove elements based on a condition in this rdd. Let's say we want to remove all the elements that does not have even number of digits in our rdd we can do it as follows:

In [12]:
even_digits_rdd = total_digits_rdd.filter(lambda x: x[1] % 2 == 0)
even_digits_rdd.take(3)

[(16, 2), (25, 2), (36, 2)]

If we want to flip the pairs, so that we have number of digits paired to a number we can do it as follows:

In [13]:
flipped_pairs_rdd = even_digits_rdd.map(lambda x: (x[1], x[0]))
flipped_pairs_rdd.take(3)

[(2, 16), (2, 25), (2, 36)]

We can use the `groupByKey` method to group elements in an rdd based on a key. Here is an example.

In [14]:
grouped_rdd = flipped_pairs_rdd.groupByKey()
grouped_rdd.take(3)

[(2, <pyspark.resultiterable.ResultIterable at 0x7e455c3f1ff0>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7e455c3f3610>),
 (6, <pyspark.resultiterable.ResultIterable at 0x7e455c3f3820>)]

We can see that our keys are correctly paired however the value is a `pyspark.resultiterable.ResultIterable` object. We can use the map function to convert that to a list.

In [16]:
grouped_and_mapped_to_list_rdd = grouped_rdd.map(lambda x: (x[0], list(x[1])))
grouped_and_mapped_to_list_rdd.take(1)

[(2, [16, 25, 36, 49, 64, 81])]

We can return the key and the avarage of it's values, for that we can use the `map` function to do that.

In [17]:
avaraged_rdd = grouped_and_mapped_to_list_rdd.map(lambda x: (x[0], sum(x[1])/len(x[1])))
avaraged_rdd.collect()

[(2, 45.166666666666664),
 (4, 4675.5),
 (6, 471838.0),
 (8, 47204941.666666664),
 (10, 4720705565.0)]